In [1]:
from utils import *
from train import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def show_image(img, ax=None):
    if ax is None:
        ax = plt
    if len(img.shape) == 2:
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img)

In [3]:
#model = training('large-data/vehicles', 'large-data/non-vehicles')
model = Model('model.pkl')

In [4]:
test_image = read_image('test_images/test1.jpg')

search = CarSearch(model)
annotated = search.annotate_cars_in_image(test_image)
show_image(annotated)

ValueError: operands could not be broadcast together with shapes (1,9484) (8460,) (1,9484) 

## Video

In [5]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

count = [0]
def save_image(img):
    mpimg.imsave('output-images/image-%d.png'%count[0], img)
    count[0] += 1
    return img

def process_video(video_filename):
    model = Model('model.pkl')
    search = CarSearch(model) 
    output = "annotated_P5_" + video_filename
    input_clip = VideoFileClip(video_filename).subclip(5,11)
    clip = input_clip.fl_image(search.annotate_cars_in_video)
    #clip = input_clip.fl_image(save_image)
    %time clip.write_videofile(output, audio=False)

v1 = 'test_video.mp4'
v2 = 'project_video.mp4'

process_video(v2)

[MoviePy] >>>> Building video annotated_P5_project_video.mp4
[MoviePy] Writing video annotated_P5_project_video.mp4


 99%|█████████▉| 150/151 [01:13<00:00,  2.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: annotated_P5_project_video.mp4 

CPU times: user 2min, sys: 11.8 s, total: 2min 12s
Wall time: 1min 15s


In [ ]:
count

In [ ]:
'output_images/image-%d.jpg'%count[0]

In [ ]:
mpimg